In [32]:
# attempt 2

filename = "warped.png"
with open(filename, "rb") as f:
    data = f.read()

# break into 1024-byte chunks
CHUNK_SIZE = 1024
chunks = [data[i:i+CHUNK_SIZE] for i in range(0, len(data), CHUNK_SIZE)]

print(f"Total chunks: {len(chunks)}")

Total chunks: 1533


In [ ]:
# filter out empty chunks (NO don't do this you need them!)
# chunks = [chunk for chunk in chunks if chunk.strip(b"\x00")]
# print(f"Filtered chunks: {len(chunks)}")

In [34]:
starts = [i for i, chunk in enumerate(chunks) if chunk.startswith(b'\x89PNG')]
print(f"PNG headers found at: {starts}")
ends = [i for i, chunk in enumerate(chunks) if b'IEND' in chunk]
print(f"PNG IEND found at: {ends}")
headers = [i for i, chunk in enumerate(chunks) if b'IHDR' in chunk]
print(f"PNG IHDR found at: {headers}")

PNG headers found at: [0, 1, 2]
PNG IEND found at: [721, 797, 1530]
PNG IHDR found at: [0, 1, 2]


In [35]:
start_chunks = [chunks[i] for i in starts]
ends_chunks = [chunks[i] for i in ends]
middle_chunks = [chunks[i] for i in range(starts[0]+1, ends[0]) if i not in starts and i not in ends]

In [36]:
print(721 % 3)
print(797 % 3)
print(1530 % 3)

1
2
0


In [37]:
# trying to reassemble the PNGs
os.makedirs("reassembled_pngs", exist_ok=True)

# assume i % 3 belongs to png i
png0 = [chunk for i, chunk in enumerate(chunks) if i % 3 == 0]
png1 = [chunk for i, chunk in enumerate(chunks) if i % 3 == 1]
png2 = [chunk for i, chunk in enumerate(chunks) if i % 3 == 2]

In [38]:
len(png0), len(png1), len(png2)

(511, 511, 511)

In [39]:
png0[-1]

b'$\xf6\xdb"\x02\x86\xe8\xc1\xa8\xb4\xb7\xf4h\x91\x11\xbd\x9f\x9f#G\xaa\x94\x070\xb6\xe0\x96\xec\x81\x9d\x932\xd6\x9b\xfa)d\r\x9d\x01nw\xed[\xc6\x93l+F\x9b>\xcd\x08\x8cs\x8c*\xfd\xc4\xa8[\xde\xa7f)}\xc4\x10\xc8\xb5Ml\xcd\x03\x9f?\xdf\xff\xc2\xb3\xce\xf4\xa3\xc7p\xd63m\xc7J\xbdsZl\xe9\xbew-\x19\xe3\xb3\x91\x12\x8a\xb8\xbb)\x11H\x16\xa8jV\x91G\xf1##\xbb.\x06\xe3\xe3\x00V\xe8\x8anv,\xc0p\xec\xf3|\xd1\x11\xd6[*g~\xa1\xd3\xdd\x9fv\x04p:\xae\xf5\xe1\xad\x19\x1eo\xb3\x83\x0eRK\xa5\xdbzno\xbcJ\xdf\xc0J\x02\xd6\xc8-}\xc5)x\x1e\xf4\x9a\xc2\xb3\xf4\xb79i\x88;\xad\x7f\xc7\xf9\x82\x9bU,"X.\xc1!\x00\xfd\xa9\xfbf\x0b\t\x1cDt]\x15\x1b\xe8>\xfd\xa9\xfb\xfe9YR\xb6\x87\xb2\xec\x88l!\xdb\xac\xf4\xec\xfdYr\x9dZ\xdf\xb3\r\x17m\'3\xd9z\xce\x05$^\xf0\xb4\xf9\x02\xce\xd0_d\xed\x91\xe5[\xd5\x04\xea\xe8Z\x02?\x9e@g\xbd\xfb\xa7\xda\\\t\x7f3~\xf4\n\xc1^0A\x80\x91\xecttJl\x15\xfa\xbd\xc5b\x85-+\xe8T\xb6KA^\xa6\xbfcK\xe3\xe0E\xff\xe1\x80\x97\xb6\x9d\x90\xda\xfa\x05\'\x04}\xba,Yv8\x8e\xc0gp.t\x95\xa9\xc7u8Ii;:\x9c\x8

In [40]:
for i, chunk in enumerate(png0):
    if not chunk.strip(b"\x00"):
        print(f"chunk {i} is empty")

In [45]:
for i, chunk in enumerate(png1):
    if b'IEND' in chunk:
        print(f"chunk {i} is the IEND chunk")
    if not chunk.strip(b"\x00"):
        # print(f"chunk {i} is empty")
        continue

chunk 240 is the IEND chunk


In [44]:
for i, chunk in enumerate(png2):
    if b'IEND' in chunk:
        print(f"chunk {i} is the IEND chunk")
    if not chunk.strip(b"\x00"):
        # print(f"chunk {i} is empty")
        continue

chunk 265 is the IEND chunk


In [43]:
# join the chunks back together
reassembled_png0 = b''.join(png0)
reassembled_png1 = b''.join(png1)
reassembled_png2 = b''.join(png2)

with open("reassembled_pngs/reassembled_0.png", "wb") as f:
    f.write(reassembled_png0)
with open("reassembled_pngs/reassembled_1.png", "wb") as f:
    f.write(reassembled_png1)
with open("reassembled_pngs/reassembled_2.png", "wb") as f:
    f.write(reassembled_png2)